In [12]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import Dataset

from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [65]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 50

# Architecture
NUM_FEATURES = 28*28
NUM_CLASSES = 345

# Other
DEVICE = "cuda:0"
GRAYSCALE = True

In [4]:
# Load data
images = np.load("Data/sample_1000_image.npy")
labels = np.load("Data/sample_1000_label.npy")

# Normalize image data.  0-255 to 0-1
images = images / 255
df = pd.DataFrame(np.concatenate((images, labels), axis=1))

# Rename the last column as "label"
df.rename(columns={784:"label"}, inplace=True)

# Convert label column to integer type
df['label'] = df['label'].astype('int64')

In [6]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [7]:
# Get "img" data frame and "lbl" series from the df
img = df.iloc[:, 0:-1]
lbl = df['label']
# Split into train, validation and test set
x_train1, x_test, y_train1, y_test = train_test_split(img, lbl, test_size = 0.10, random_state = 123, stratify = lbl)
x_train, x_valid, y_train, y_valid = train_test_split(x_train1, y_train1, test_size = 0.2, random_state = 123, stratify = y_train1)

In [9]:
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)
print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(248400, 784)
(62100, 784)
(34500, 784)
(248400,)
(62100,)
(34500,)


In [10]:
# Convert dataframe to tensor
x_train = torch.tensor(x_train.values)
y_train = torch.tensor(y_train.values)

x_valid = torch.tensor(x_valid.values)
y_valid = torch.tensor(y_valid.values)

x_test = torch.tensor(x_test.values)
y_test = torch.tensor(y_test.values)

In [49]:
# Reshape all tensors to the size of (length, 1, 28, 28)
x_train = x_train.reshape((-1, 1, 28, 28))
x_valid = x_valid.reshape((-1, 1, 28, 28))
x_test = x_test.reshape((-1, 1, 28, 28))

In [50]:
# Create dataloaders

train_dataset = TensorDataset(x_train, y_train)
valid_dataset = TensorDataset(x_valid, y_valid)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True, num_workers=4)

valid_loader = DataLoader(dataset=valid_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False, num_workers=4)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False, num_workers=4)

In [51]:
# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([128, 1, 28, 28])
Image label dimensions: torch.Size([128])


In [52]:
device = torch.device(DEVICE if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

num_epochs = 2
for epoch in range(num_epochs):

    for batch_idx, (x, y) in enumerate(train_loader):
        
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(device)
        y = y.to(device)
        break

Epoch: 1 | Batch index: 0 | Batch size: 128
Epoch: 2 | Batch index: 0 | Batch size: 128


In [53]:

##########################
### MODEL
##########################


class LeNet5(nn.Module):

    def __init__(self, num_classes, grayscale=False):
        super(LeNet5, self).__init__()
        
        self.grayscale = grayscale
        self.num_classes = num_classes

        if self.grayscale:
            in_channels = 1
        else:
            in_channels = 3

        self.features = nn.Sequential(
            
            nn.Conv2d(in_channels, 6, kernel_size=5),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.MaxPool2d(kernel_size=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.Linear(120, 84),
            nn.Linear(84, num_classes),
        )


    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas

In [54]:
torch.manual_seed(RANDOM_SEED)

model = LeNet5(NUM_CLASSES, GRAYSCALE)
model = model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [59]:
DEVICE

'cuda:0'

In [55]:
def print_sizes(self, input, output):

    print('Inside ' + self.__class__.__name__ + ' forward')
    print('input size:', input[0].size())
    print('output size:', output.data.size())

    
## Debugging


# model.features[0].register_forward_hook(print_sizes)
# model.features[1].register_forward_hook(print_sizes)
# model.features[2].register_forward_hook(print_sizes)
# model.features[3].register_forward_hook(print_sizes)

# model.classifier[0].register_forward_hook(print_sizes)
# model.classifier[1].register_forward_hook(print_sizes)
# model.classifier[2].register_forward_hook(print_sizes)


In [61]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
        
        features = features.float()
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [66]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.float()
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 500:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))


    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d | Train: %.3f%% | Validation: %.3f%%' % (
              epoch+1, NUM_EPOCHS, 
              compute_accuracy(model, train_loader, device=DEVICE),
              compute_accuracy(model, valid_loader, device=DEVICE) ))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/050 | Batch 0000/1941 | Cost: 2.3292
Epoch: 001/050 | Batch 0500/1941 | Cost: 2.3485
Epoch: 001/050 | Batch 1000/1941 | Cost: 2.2854
Epoch: 001/050 | Batch 1500/1941 | Cost: 2.6006
Epoch: 001/050 | Train: 46.968% | Validation: 45.087%
Time elapsed: 0.45 min
Epoch: 002/050 | Batch 0000/1941 | Cost: 2.0632
Epoch: 002/050 | Batch 0500/1941 | Cost: 2.1191
Epoch: 002/050 | Batch 1000/1941 | Cost: 2.2670
Epoch: 002/050 | Batch 1500/1941 | Cost: 2.5054
Epoch: 002/050 | Train: 46.670% | Validation: 44.976%
Time elapsed: 0.89 min
Epoch: 003/050 | Batch 0000/1941 | Cost: 2.4445
Epoch: 003/050 | Batch 0500/1941 | Cost: 2.3048
Epoch: 003/050 | Batch 1000/1941 | Cost: 2.5274
Epoch: 003/050 | Batch 1500/1941 | Cost: 2.5672
Epoch: 003/050 | Train: 47.178% | Validation: 45.314%
Time elapsed: 1.34 min
Epoch: 004/050 | Batch 0000/1941 | Cost: 2.4236
Epoch: 004/050 | Batch 0500/1941 | Cost: 2.3715
Epoch: 004/050 | Batch 1000/1941 | Cost: 2.6210
Epoch: 004/050 | Batch 1500/1941 | Cost: 2.5402
E

Epoch: 031/050 | Batch 1500/1941 | Cost: 2.5402
Epoch: 031/050 | Train: 49.205% | Validation: 46.589%
Time elapsed: 13.95 min
Epoch: 032/050 | Batch 0000/1941 | Cost: 2.1589
Epoch: 032/050 | Batch 0500/1941 | Cost: 2.5443
Epoch: 032/050 | Batch 1000/1941 | Cost: 2.2971
Epoch: 032/050 | Batch 1500/1941 | Cost: 2.1511
Epoch: 032/050 | Train: 49.343% | Validation: 46.638%
Time elapsed: 14.39 min
Epoch: 033/050 | Batch 0000/1941 | Cost: 2.5361
Epoch: 033/050 | Batch 0500/1941 | Cost: 2.2396
Epoch: 033/050 | Batch 1000/1941 | Cost: 2.5025
Epoch: 033/050 | Batch 1500/1941 | Cost: 2.5259
Epoch: 033/050 | Train: 49.548% | Validation: 46.692%
Time elapsed: 14.84 min
Epoch: 034/050 | Batch 0000/1941 | Cost: 2.4816
Epoch: 034/050 | Batch 0500/1941 | Cost: 2.2031
Epoch: 034/050 | Batch 1000/1941 | Cost: 2.4419
Epoch: 034/050 | Batch 1500/1941 | Cost: 2.5499
Epoch: 034/050 | Train: 49.430% | Validation: 46.515%
Time elapsed: 15.28 min
Epoch: 035/050 | Batch 0000/1941 | Cost: 2.1476
Epoch: 035/050 |

In [64]:
# Accuracy after 10 epochs
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

Test accuracy: 44.30%


In [67]:
# Accuracy after 60 epochs
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

Test accuracy: 46.79%
